In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

The following code creates a time series dataset suitable for creating a forecast of booked rooms by hotel, assigned room type. The idea being that summary information is great, but can't make any decisions with it. I wanted to answer the question of what is the room utilization by day? 

I don't know the total available room count, but graphs of the output show clear plateaus, so will be able to guess at total capacity. That analysis to follow.

In [ ]:
import numpy as np
import pandas as pd
from pandas.core.dtypes import dtypes
from pandas.core.indexes.base import Index
from pandas.core.indexes.range import RangeIndex 
from pandasql import sqldf
import datetime
import time 
from datetime import date 
from pandas.tseries.offsets import DateOffset

hotel_booking = pd.read_csv('../input/hotel-booking/hotel_booking.csv', 
    dtype = {'stays_in_weekend_nights':np.float64, 'stays_in_week_nights':np.float64}, parse_dates=True, infer_datetime_format=True )
dates =pd.DataFrame(data = pd.date_range('6/1/2015' , '8/31/2017' ) )





CheckOuts = hotel_booking[hotel_booking["reservation_status"] == 'Check-Out']
codt = CheckOuts.dtypes

#codt 


Checkouts2 = sqldf('select *,  name || [phone-number] as bookId from CheckOuts')
Checkouts2['reservation_status_date'] = pd.to_datetime(Checkouts2['reservation_status_date'])
Checkouts2['stays_in_weekend_nights'] = pd.to_numeric(Checkouts2['stays_in_weekend_nights'])
Checkouts2['stays_in_week_nights'] = pd.to_numeric(Checkouts2['stays_in_week_nights'])

weekend=Checkouts2['stays_in_weekend_nights']

weeknight = Checkouts2['stays_in_week_nights']

for i in Checkouts2.index.values:
   
    Checkouts2.loc[i,'Arrival_Date'] = Checkouts2.loc[i,'reservation_status_date'] - pd.DateOffset(days = Checkouts2.loc[i,'stays_in_weekend_nights'] + Checkouts2.loc[i,'stays_in_week_nights'])

Checkouts2['Arrival_Date'] = pd.to_datetime(Checkouts2['Arrival_Date'])
Checkouts2['reservation_status_date'] = pd.to_datetime(Checkouts2['reservation_status_date'])
test= sqldf('select [0] as Date from dates')

test['Date'] = pd.to_datetime(test['Date'])


test2 = sqldf('select c2.hotel, c2.assigned_room_type, d.Date, c2.Arrival_Date, 1 as [Count] from test d left outer join Checkouts2 c2 on d.Date >= c2.Arrival_Date and d.Date <= reservation_status_date')


test2

test2.to_csv('./hotel_booking_output_test1.csv')
